    This requires that the Xlaunch server is running. Make sure to check “Disable access control”
    reference  https://www.gregbrisebois.com/posts/chromedriver-in-wsl2/

In [1]:
import os
import sys
import pandas as pd
os.path.dirname(sys.executable)

'/home/mbrenes/anaconda3/envs/disney/bin'

In [1]:
def getData(payload):
    
    date = payload.text.split('#')[0].strip()
    split2list = payload.text.split('#')[1].split('\n')
    order_num =split2list[0]
    del split2list[0]
    Amount = float(split2list[-1].replace("$", "").strip())
    del split2list[-1]
    children = len(split2list)
    data = []
    for kid in split2list:
        if children > 1:
            new_amount = Amount / children
            data.append([date,order_num,kid,new_amount])
        else:
            data.append([date,order_num,kid,Amount])
    return data    

In [2]:
from time import sleep
from selenium import webdriver
# this is tested on Firefox or you can use "webdriver.Chrome()"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
# chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)

NameError: name 'Options' is not defined

In [3]:
# open Browser and navigate to url
browser = webdriver.Chrome()
browser.get('https://ccsdxspandedlearning.revtrak.net/account#')
sleep(3)

WebDriverException: Message: unknown error: Chrome failed to start: crashed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


In [4]:
username= browser.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[1]/form/input[1]')
password = browser.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[1]/form/input[2]')
submit = browser.find_element_by_css_selector(".loginButton")
submit = browser.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[1]/form/button')
username.send_keys("house@brenesfinancial.com")
password.send_keys("Krissy14")
# Step 4) Click Login

sleep(7)

In [267]:
# extract data save to dataframe
records=[]
col = ['Date','Order_id','Child','Amount']
# pagination loop
paginate = '/html/body/div[4]/div/div[2]/div/div[1]/dir-pagination-controls/ul/li[7]/a'
# get number of pages
num_pages = int(browser.find_elements_by_xpath(paginate)[0].text)
next_button = '/html/body/div[4]/div/div[2]/div/div[1]/dir-pagination-controls/ul/li[8]/a'
# click the button num_pages times
current_page = 1
for p in range (num_pages):
    print("Page " + str(current_page)+ " of "+ str(num_pages))
    if current_page == 1:
        pass
    else:
        browser.find_element_by_xpath(next_button).click()
        time.sleep(3)
    ids = browser.find_elements_by_xpath('//*[@id]')
    for ii in ids:
        if 'order_' in ii.get_attribute('id'):
            order = ii.get_attribute('id')
            payload = browser.find_element_by_id(order)
            for rec in getData(payload):
                records.append(rec)
    current_page+=1
# end pagination loop          
df = pd.DataFrame(records, columns = col)
df['Child']= df['Child'].str.split(',', expand=True)[1]
df['Year']= df['Date'].str.split(' ',expand=True)[2]
Final = df.groupby(['Child','Year']).sum()
Final

Page 1 of 7
Page 2 of 7
Page 3 of 7
Page 4 of 7
Page 5 of 7
Page 6 of 7
Page 7 of 7


Amount
Child    Year             
 Krissy  2019  1415.000000
         2020  1245.000000
         2021  3876.333333
         2022    85.000000
 Mikayla 2021  1349.666667
         2022   165.000000

In [268]:
browser.close()